# XGB - Hyperparameter Tuning

## Setup

### Environment Variables

In [1]:
%env TOKENIZERS_PARALLELISM=true

env: TOKENIZERS_PARALLELISM=true


### Import Libraries

In [1]:
import os
import random
from typing import Tuple, List
import warnings
import pickle as pkl
import torch

from dotenv import load_dotenv
import wandb
from tqdm import tqdm
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from lightgbm import log_evaluation, early_stopping
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import cohen_kappa_score

In [3]:
os.chdir("../../")
warnings.simplefilter('ignore')
load_dotenv()

True

In [4]:
from lib.model.utils import quadratic_weighted_kappa, qwk_obj, get_score

### Paths

In [5]:
class Paths:
    # Competition data with added topic column
    train_csv: str = "data/processed/train.csv"
    test_csv: str = "data/processed/test.csv"
    feature_csv: str = "data/feature_engg/all_features.csv"

    # Output path
    output_path: str = "output/model_dir_ht"
    model_path: str = os.path.join(output_path, "{model_name}")
    tokenizer_path: str = os.path.join(model_path, "{model_name}_tokenizer")
    threshold_path: str = os.path.join(model_path, "threshold.pkl")
    logging_path: str = os.path.join(model_path, "logging")

### Configurations

In [6]:
class CFG:
    num_labels: int = 6
    seed: int = 29
    lgbm_a: float = 2.998
    lgbm_b: float = 1.092
    lgbm_n_folds: int = 7

### Setting Random Seed

In [7]:
def seed_everything() -> None:
    """Seed everything to ensure reproducibility

    Sources:
    1. https://www.kaggle.com/code/alejopaullier/aes-2-multi-class-classification-train
    2. https://www.kaggle.com/code/hashidoyuto/deberta-baseline-aes2-0-train
    """
    random.seed(CFG.seed)
    os.environ["PYTHONHASHCFG.SEED"] = str(CFG.seed)
    np.random.seed(CFG.seed)
    torch.manual_seed(CFG.seed)
    torch.cuda.manual_seed(CFG.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything()

### Sweep Configuration

In [8]:
sweep_config = {
    # How to perform hyperparameter tuning
    "method": "random",
    # How to evaluate which hyperparameter combination is good
    "metric": {
        "name": "QWK",
        "goal": "maximize",
    },
    # Hyperparameters to tune
    "parameters": {
        "learning_rate": {"distribution": "uniform", "min": 0.01, "max": 0.1},
        "max_depth": {"distribution": "int_uniform", "min": 3, "max": 10},
        "num_leaves": {"distribution": "int_uniform", "min": 5, "max": 25},
        "n_estimators": {"distribution": "int_uniform", "min": 100, "max": 2000},
        "reg_alpha": {"distribution": "uniform", "min": 0.0, "max": 1.0},
        "reg_lambda": {"distribution": "uniform", "min": 0.0, "max": 1.0},
        "colsample_bytree": {"distribution": "uniform", "min": 0.0, "max": 1.0},
    },
}

### WandB setup

In [9]:
WANDB_PROJECT = "Kaggle_ASE_2.0"

In [10]:
wandb.login(key=os.environ.get('WANDB_API_KEY'))
sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT)

wandb: Currently logged in as: shakleenishfar (laplacesdemon43). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ishfar/.netrc


Create sweep with ID: cetfas2j
Sweep URL: https://wandb.ai/laplacesdemon43/Kaggle_ASE_2.0/sweeps/cetfas2j


## Data Preparation

In [11]:
df = pd.read_csv(Paths.train_csv, usecols=["score", "topic"])
X = pd.read_csv(Paths.feature_csv)
ids, X = X.loc[:, "essay_id"], X.drop(columns=["essay_id"])
topic, y = df.loc[:, "topic"], df.loc[:, "score"]

del df
X.shape, y.shape

((17307, 104), (17307,))

## Training

In [12]:
a = CFG.lgbm_a
b = CFG.lgbm_b

In [13]:
callbacks = [
    # log_evaluation(period=25),
    early_stopping(stopping_rounds=75, first_metric_only=True),
]

### Utility Functions

In [14]:
def validate_model(idx, X_valid, y_valid, id_valid, predictor):
    y_valid_int = y_valid.astype(np.int64).to_numpy()
    preds_raw = predictor.predict(X_valid) + a
    preds = preds_raw.clip(1, 6).round().astype(np.int64)

    oof = pd.DataFrame(
        {
            "essay_id": id_valid,
            "score": y_valid_int - 1,
            "pred_score": preds - 1,
            "raw": preds_raw
        }
    )

    return oof

In [15]:
def train_loop(sweep_config, X_train, y_train, X_valid, y_valid):
    y_train = y_train.astype(np.float32) - a
    y_valid = y_valid.astype(np.float32) - a

    model = lgb.LGBMRegressor(
        objective=qwk_obj,
        metrics="None",
        learning_rate=sweep_config.learning_rate,
        max_depth=sweep_config.max_depth,
        num_leaves=sweep_config.num_leaves,
        colsample_bytree=sweep_config.colsample_bytree,
        reg_alpha=sweep_config.reg_alpha,
        reg_lambda=sweep_config.reg_lambda,
        n_estimators=sweep_config.n_estimators,
        random_state=CFG.seed,
        extra_trees=True,
        class_weight="balanced",
        n_jobs=6,
        verbosity=0,
    )

    # Train model
    predictor = model.fit(
        X_train,
        y_train,
        eval_names=["train", "valid"],
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks,
    )

    return model, predictor

### Training Model

In [16]:
def main(sweep_config=None):
    with wandb.init(config=sweep_config):
        sweep_config = wandb.config
        oof_df = pd.DataFrame()

        skf = StratifiedGroupKFold(n_splits=7, random_state=CFG.seed, shuffle=True)

        for idx, (train_idx, valid_idx) in enumerate(skf.split(X, y, groups=topic)):
            X_train, y_train = X.loc[train_idx], y.loc[train_idx]
            X_valid, y_valid = X.loc[valid_idx], y.loc[valid_idx]
            id_valid = ids.loc[valid_idx].to_numpy()

            _, predictor = train_loop(sweep_config, X_train, y_train, X_valid, y_valid)

            oof = validate_model(idx, X_valid, y_valid, id_valid, predictor)
            oof_df = pd.concat([oof_df, oof])

        score = get_score(oof_df["score"], oof_df["pred_score"])
        print(f"Overall QWK Score: {score}")
        wandb.log({"QWK": score})

### Initiate Sweep

In [17]:
wandb.agent(sweep_id, main, count=50, project=WANDB_PROJECT)

wandb: Agent Starting Run: 84ek4yo9 with config:
wandb: 	class_weight: None
wandb: 	colsample_bytree: 0.32921927464812795
wandb: 	learning_rate: 0.026197564003735213
wandb: 	max_depth: 8
wandb: 	n_estimators: 857
wandb: 	num_leaves: 23
wandb: 	reg_alpha: 0.2913564174324196
wandb: 	reg_lambda: 0.8484726778692203


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
Early stopping, best iteration is:
[92]	train's QWK: 0.775404	valid's QWK: 0.721177
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
Early stopping, best iteration is:
[291]	train's QWK: 0.808024	valid's QWK: 0.801672
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
Early stopping, best iteration is:
[480]	train's QWK: 0.815588	valid's QWK: 0.818471
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
Early stopping, best iteration is:
[766]	train's QWK: 0.834219	valid's QWK: 0.746194
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rou

QWK,▁
QWK,0.78501


wandb: Agent Starting Run: 0jo3y40d with config:
wandb: 	class_weight: balanced
wandb: 	colsample_bytree: 0.23599436533860307
wandb: 	learning_rate: 0.07610723203302736
wandb: 	max_depth: 7
wandb: 	n_estimators: 385
wandb: 	num_leaves: 7
wandb: 	reg_alpha: 0.02364107985416863
wandb: 	reg_lambda: 0.39830484260027466


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
Early stopping, best iteration is:
[50]	train's QWK: 0.773534	valid's QWK: 0.726214
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
Did not meet early stopping. Best iteration is:
[383]	train's QWK: 0.810347	valid's QWK: 0.803172
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
Did not meet early stopping. Best iteration is:
[384]	train's QWK: 0.80943	valid's QWK: 0.820576
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
Did not meet early stopping. Best iteration is:
[364]	train's QWK: 0.815089	valid's QWK: 0.73569
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until valid

...

[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

QWK,▁
QWK,0.78973


wandb: Agent Starting Run: 6l5c50v3 with config:
wandb: 	class_weight: None
wandb: 	colsample_bytree: 0.6558306691334778
wandb: 	learning_rate: 0.055542764546451566
wandb: 	max_depth: 3
wandb: 	n_estimators: 1222
wandb: 	num_leaves: 8
wandb: 	reg_alpha: 0.4486192441650094
wandb: 	reg_lambda: 0.16759770631993398


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[37]	train's QWK: 0.780408	valid's QWK: 0.69323
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

QWK,▁
QWK,0.79053


## Wrapping up

In [18]:
wandb.finish()